In [54]:
import os, time
import regex as re
import pandas as pd
from dotenv import load_dotenv
from map_industries import make_ind_dict, make_tag_dict
from write_properties import create_company_property
from write_companies import update_company, batch_update_company
from industry_grouping import main, explode_df

load_dotenv()

DBDIR = "C:/Users/galon/Sputnik ATX Team Dropbox/Programming Datasets"
RAW_DIR = os.path.join(DBDIR, "data", "raw")
CLEAN_DIR = os.path.join(DBDIR, "data", "clean")
IND_DIR = os.path.join(CLEAN_DIR, "industry_mapping")
VC_INDUSTRY_COLS = os.path.join(RAW_DIR, "hs", "vc_industry_columns.csv")
STARTPATH = os.path.join(CLEAN_DIR, 'scraped_data', 'crunchbase', 'cb_starts_main_scraped.csv')
INVPATH = os.path.join(CLEAN_DIR, 'scraped_data', 'crunchbase', 'cb_vc_main_merged.csv')
MAPPING_OUTPATH = os.path.join(IND_DIR, "cb_starts_mapped.csv")
START_HERF_PATH = os.path.join(CLEAN_DIR, 'scraped_data', 'crunchbase','cb_inv_overview_scraped.csv')


### Column inpaths

In [52]:
def make_inv_focus_dict(column):

    df = main()

    return [
        {
            "id": str(k),
            "properties": {
                'inv_inds': re.sub(r"\s", "_", str(d1[new_col].values[n])).lower(),
                "name": str(df["Company name"].values[n]),
            },
        }
        for n, k in enumerate(d1["Record ID"].values)
    ]

In [55]:
pd.read_csv(INVPATH)

,Unnamed: 0,num_emp,investor_type,cb_stages,hub_tags,Active,founded,legal_name,founders,industries,...,MAX Round Size,MIN Check Size,MIN Round Size,Company ID,min_check,max_check,max_params,min_params,abs_max,abs_min
0,0,1-10,Venture Capital,"Early Stage Venture, Late Stage Venture, Seed",Crunchbase Venture Program,Active,2013,"Prelude Ventures, LLC","Nat Simons, ​Laura Baxter-Simons","Energy Efficiency, Energy Management, Energy S...",...,NaN,NaN,NaN,9.060134e+09,NaN,NaN,"(6500000.0, nan, nan)","(10100000.0, nan, nan)",6500000.0,10100000.0
1,1,11-50,Venture Capital,NaN,NaN,Active,2020,NaN,Alexandra Mihaescu,"Finance, Financial Services, FinTech, Venture ...",...,NaN,NaN,NaN,9.060154e+09,NaN,NaN,"(nan, nan, nan)","(nan, nan, nan)",NaN,NaN
2,2,1-10,Venture Capital,"Early Stage Venture, Seed",NaN,Active,2011,NaN,Igor Lutz,Financial Services,...,NaN,NaN,NaN,9.060134e+09,NaN,NaN,"(600000.0, nan, nan)","(100000.0, nan, nan)",600000.0,100000.0
3,3,1-10,Venture Capital,Early Stage Venture,Crunchbase Venture Program,Active,2018,NaN,Ollen Douglass,"Finance, Financial Services, Venture Capital",...,NaN,NaN,NaN,9.060077e+09,NaN,NaN,"(900629.0, nan, nan)","(350000.0, nan, nan)",900629.0,350000.0
4,4,11-50,Micro VC,"Early Stage Venture, Late Stage Venture",Crunchbase Venture Program,Active,2001,NaN,Greg Ennis,"Finance, Financial Services, SaaS",...,NaN,NaN,NaN,9.059204e+09,NaN,NaN,"(2960000.0, nan, nan)","(2960000.0, nan, nan)",2960000.0,2960000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1554,1554,1-10,NaN,NaN,NaN,Active,NaN,NaN,"Jerome Camp, Justin Camp",NaN,...,NaN,NaN,NaN,9.060165e+09,NaN,NaN,"(nan, nan, nan)","(nan, nan, nan)",NaN,NaN
1555,1555,11-50,NaN,NaN,NaN,Active,"Mar 28, 1930",H+ Die Spitäler der Schweiz,NaN,"Association, Business Development, Health Care...",...,NaN,NaN,NaN,9.060183e+09,NaN,NaN,"(nan, nan, nan)","(nan, nan, nan)",NaN,NaN
1556,1556,11-50,Venture Capital,"Early Stage Venture, Late Stage Venture, Seed",Crunchbase Venture Program,Active,1996,NaN,"J.B. Pritzker, Tony Pritzker","Finance, Financial Services, Venture Capital",...,NaN,NaN,NaN,9.060184e+09,NaN,NaN,"(29999995.0, nan, nan)","(1000000.0, nan, nan)",29999995.0,1000000.0
1557,1557,NaN,Venture Capital,NaN,NaN,Active,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.060028e+09,NaN,NaN,"(nan, nan, nan)","(nan, nan, nan)",NaN,NaN


In [47]:
main()

,inv_inds,href,name,description,ref
0,;Marketing;Location Based Services,/organization/digifi-7782,DigiFi,Wifi Marketing tool that helps venues to analy...,"Ad Network, Analytics, Big Data, Location Base..."
1,;FinTech,/organization/self-lender,Self Financial,Self Financial is a venture-backed fintech sta...,"Financial Services, FinTech, Internet, Persona..."
2,;Biotechnology;Pharmaceuticals,/organization/rdmd,AllStripes,AllStripes is a healthcare technology company ...,"Big Data, Biotechnology, Health Care, Market R..."
3,;IoT;Information Technology;Software,/organization/elev%C3%A1t,Elevát,Elevát is an industrial IoT platform helping m...,"Computer, Industrial, Information Technology, ..."
4,;Productivity Tools;Software,/organization/smartcuts,Smartcuts,Smartcuts develops an online browser tool that...,"Browser Extensions, Internet, Productivity Too..."
...,...,...,...,...,...
10268,,/organization/backstage-capital,Backstage Capital,Backstage Capital is a venture capital fund th...,"Finance, Financial Services, Venture Capital"
10269,;Information Technology;FinTech,/organization/bamboo-eacd,Bamboo,Bamboo is a digital investment platform that p...,"Financial Services, FinTech, Information Techn..."
10271,;Software;Payment Card Industry (PCI),/organization/spendesk,Spendesk,Spendesk is a smart spend management software ...,"B2B, Credit Cards, Financial Services, FinTech..."
10272,;IoT;Information Technology,/organization/koidra,Koidra,Koidra is an IT company that transforms agricu...,"Information Technology, Internet of Things"
